In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

## Demographic

In [2]:
df = pd.ExcelFile('Zip Code Recommendations Data.xlsx')

In [3]:
y_data = pd.read_excel(df,'InputData')

In [4]:
demographic = pd.read_csv("zip_codes_2018_5yr.csv")
demographic.head()

,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,87537,2014-01-01,2510,856,1283,1227,42.1,58,75,87,...,NaN,NaN,304,171,80,593,20,NaN,NaN,NaN
1,87017,2014-01-01,346,112,150,196,50.9,0,0,12,...,NaN,NaN,0,27,15,96,40,NaN,NaN,NaN
2,87528,2014-01-01,3505,727,1758,1747,27.9,260,194,159,...,NaN,NaN,293,266,298,1237,138,NaN,NaN,NaN
3,87533,2014-01-01,133,58,49,84,25.8,0,12,8,...,NaN,NaN,24,8,7,58,3,NaN,NaN,NaN
4,87511,2014-01-01,2896,787,1177,1719,36.0,142,63,44,...,NaN,NaN,263,107,163,1008,279,NaN,NaN,NaN


In [5]:
demographic.columns = ['zip']+list(demographic.columns[1:])
demographic = demographic.drop('do_date',axis=1)
demographic.head()

,zip,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,87537,2510,856,1283,1227,42.1,58,75,87,52,...,NaN,NaN,304,171,80,593,20,NaN,NaN,NaN
1,87017,346,112,150,196,50.9,0,0,12,10,...,NaN,NaN,0,27,15,96,40,NaN,NaN,NaN
2,87528,3505,727,1758,1747,27.9,260,194,159,121,...,NaN,NaN,293,266,298,1237,138,NaN,NaN,NaN
3,87533,133,58,49,84,25.8,0,12,8,4,...,NaN,NaN,24,8,7,58,3,NaN,NaN,NaN
4,87511,2896,787,1177,1719,36.0,142,63,44,123,...,NaN,NaN,263,107,163,1008,279,NaN,NaN,NaN


In [6]:
demographic.nunique()

zip                                  33120
total_pop                            15532
households                           10573
male_pop                             11966
female_pop                           12073
                                     ...  
in_school                             8713
in_undergrad_college                  3747
speak_only_english_at_home               0
speak_spanish_at_home                    0
speak_spanish_at_home_low_english        0
Length: 239, dtype: int64

In [7]:
demographic = demographic.dropna(axis=1,thresh=int(len(demographic)*0.5))
print(demographic.shape)
demographic.head()

(33120, 229)


,zip,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,...,occupation_production_transportation_material,occupation_sales_office,occupation_services,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college
0,87537,2510,856,1283,1227,42.1,58,75,87,52,...,NaN,NaN,NaN,NaN,NaN,304,171,80,593,20
1,87017,346,112,150,196,50.9,0,0,12,10,...,NaN,NaN,NaN,NaN,NaN,0,27,15,96,40
2,87528,3505,727,1758,1747,27.9,260,194,159,121,...,NaN,NaN,NaN,NaN,NaN,293,266,298,1237,138
3,87533,133,58,49,84,25.8,0,12,8,4,...,NaN,NaN,NaN,NaN,NaN,24,8,7,58,3
4,87511,2896,787,1177,1719,36.0,142,63,44,123,...,NaN,NaN,NaN,NaN,NaN,263,107,163,1008,279


In [8]:
demographic.shape

(33120, 229)

In [9]:
med_dict = dict(demographic.median())

In [10]:
demographic = demographic.fillna(value=med_dict,axis=0)
demographic.isnull().sum().sum()

0

In [11]:
zip_demographic = pd.DataFrame()
for i in y_data['ZipCode']:
    zip_demographic = pd.concat([zip_demographic,demographic[demographic['zip']==i]])
print(zip_demographic.shape)
zip_demographic.head()

(324, 229)


,zip,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,...,occupation_production_transportation_material,occupation_sales_office,occupation_services,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college
23357,77072,61818,18096,30827,30991,32.6,2158,2094,2785,1572,...,4590.0,5803.0,7922.0,5613.0,5803.0,3593,4176,4141,17999,4044
27034,77083,79996,22434,38455,41541,33.5,2639,2851,2901,1920,...,5462.0,9690.0,9782.0,9531.0,9690.0,4432,4777,5122,24503,6658
16633,77587,17040,4615,9103,7937,28.7,1005,862,930,411,...,1463.0,1177.0,1472.0,894.0,1177.0,1461,1212,1052,5092,889
10124,77002,13571,3739,11525,2046,32.7,73,0,17,41,...,105.0,696.0,218.0,2886.0,696.0,0,5,171,842,308
11373,77003,9876,4544,5108,4768,33.3,302,221,352,83,...,531.0,997.0,945.0,2819.0,997.0,349,411,280,2366,670


In [12]:
correlation = abs(zip_demographic.corr())

In [13]:
corr = correlation.copy()
corr.head()

,zip,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,...,occupation_production_transportation_material,occupation_sales_office,occupation_services,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college
zip,1.000000,0.029743,0.016000,0.028624,0.030696,0.113689,0.089805,0.082785,0.062806,0.069952,...,0.209253,0.050157,0.019459,0.090344,0.050157,0.081818,0.065441,0.076974,0.034989,0.051704
total_pop,0.029743,1.000000,0.963111,0.997820,0.997985,0.289215,0.923094,0.927036,0.918331,0.909182,...,0.787742,0.951247,0.832116,0.794571,0.951247,0.933360,0.928588,0.921773,0.957634,0.585985
households,0.016000,0.963111,1.000000,0.962337,0.959896,0.229058,0.863791,0.835959,0.817592,0.811140,...,0.707966,0.943420,0.803681,0.851237,0.943420,0.836650,0.828326,0.823027,0.881277,0.558271
male_pop,0.028624,0.997820,0.962337,1.000000,0.991623,0.301526,0.924767,0.924018,0.913752,0.903076,...,0.792113,0.945196,0.840338,0.787278,0.945196,0.927732,0.922559,0.913739,0.951965,0.584207
female_pop,0.030696,0.997985,0.959896,0.991623,1.000000,0.276190,0.917690,0.926127,0.918959,0.911315,...,0.780301,0.953154,0.820791,0.798317,0.953154,0.934934,0.930568,0.925710,0.959149,0.585285


In [14]:
k = 0
for col in zip_demographic.columns[1:]:
    try:
        l = list(corr.columns[(corr.loc[col]>0.8) & (corr.loc[col]<1)])
        if len(l)!=0:
            corr = corr.drop(l,axis=1)
            corr = corr.drop(l,axis=0)
            print(f' {k+1} iteration removed columns are : {l}\n ')
            k+=1
    except:
        pass

 1 iteration removed columns are : ['households', 'male_pop', 'female_pop', 'male_under_5', 'male_5_to_9', 'male_10_to_14', 'male_15_to_17', 'male_30_to_34', 'male_35_to_39', 'male_40_to_44', 'male_45_to_49', 'male_50_to_54', 'male_55_to_59', 'female_under_5', 'female_5_to_9', 'female_10_to_14', 'female_15_to_17', 'female_25_to_29', 'female_30_to_34', 'female_35_to_39', 'female_40_to_44', 'female_45_to_49', 'female_50_to_54', 'female_55_to_59', 'female_60_to_61', 'female_62_to_64', 'female_65_to_66', 'female_67_to_69', 'population_1_year_and_over', 'population_3_years_over', 'pop_16_over', 'pop_25_years_over', 'pop_25_64', 'not_hispanic_pop', 'income_50000_59999', 'income_60000_74999', 'income_75000_99999', 'income_100000_124999', 'pop_determined_poverty_status', 'housing_units', 'occupied_housing_units', 'dwellings_1_units_detached', 'married_households', 'family_households', 'children', 'owner_occupied_housing_units', 'mortgaged_housing_units', 'families_with_young_children', 'two_pa

In [15]:
corr.shape

(60, 60)

In [16]:
zip_demographic = zip_demographic[corr.columns]
zip_demographic.head()

,zip,total_pop,median_age,male_18_to_19,male_22_to_24,male_65_to_66,male_75_to_79,male_80_to_84,male_85_and_over,white_pop,...,commuters_by_public_transportation,commuters_by_subway_or_elevated,group_quarters,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,armed_forces,employed_agriculture_forestry_fishing_hunting_mining,employed_information,employed_public_administration
23357,77072,61818,32.6,841,1491,561,337,231,191,3962,...,1286.0,0.0,168,1801,867,2012,0.0,283.0,507.0,462.0
27034,77083,79996,33.5,1197,2218,786,489,209,204,6750,...,710.0,0.0,37,1323,626,2812,0.0,562.0,396.0,1753.0
16633,77587,17040,28.7,326,334,122,165,81,77,1353,...,104.0,10.0,9,542,293,703,0.0,57.0,20.0,154.0
10124,77002,13571,32.7,623,1156,127,42,29,25,4786,...,315.0,93.0,8395,84,378,929,0.0,384.0,80.0,96.0
11373,77003,9876,33.3,44,187,28,65,0,0,1873,...,373.0,7.0,48,317,196,286,0.0,261.0,41.0,194.0


In [17]:
y_data.columns = ['zip','Liking']
y_data.head(5)

,zip,Liking
0,77072,0.0
1,77083,0.0
2,77587,0.0
3,77002,1.0
4,77003,1.0


In [18]:
zip_demographic = zip_demographic.merge(y_data)
zip_demographic.head()

,zip,total_pop,median_age,male_18_to_19,male_22_to_24,male_65_to_66,male_75_to_79,male_80_to_84,male_85_and_over,white_pop,...,commuters_by_subway_or_elevated,group_quarters,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,armed_forces,employed_agriculture_forestry_fishing_hunting_mining,employed_information,employed_public_administration,Liking
0,77072,61818,32.6,841,1491,561,337,231,191,3962,...,0.0,168,1801,867,2012,0.0,283.0,507.0,462.0,0.0
1,77083,79996,33.5,1197,2218,786,489,209,204,6750,...,0.0,37,1323,626,2812,0.0,562.0,396.0,1753.0,0.0
2,77587,17040,28.7,326,334,122,165,81,77,1353,...,10.0,9,542,293,703,0.0,57.0,20.0,154.0,0.0
3,77002,13571,32.7,623,1156,127,42,29,25,4786,...,93.0,8395,84,378,929,0.0,384.0,80.0,96.0,1.0
4,77003,9876,33.3,44,187,28,65,0,0,1873,...,7.0,48,317,196,286,0.0,261.0,41.0,194.0,1.0


In [19]:
zip_demographic.shape

(324, 61)

In [20]:
zip_demographic['Liking'].value_counts()

1.0    13
0.0     3
Name: Liking, dtype: int64

In [21]:
# zip_demographic.to_csv("Cleaned_DemoGraphic_data_2018.csv")